# Broadcasting rules

### Imports:

In [1]:

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

print("TF version:", tf.__version__)
print("TFP version:", tfp.__version__)

TF version: 2.4.1
TFP version: 0.12.1


In [2]:
import numpy as np

## Operations on arrays of different sizes in numpy

Numpy operations can be applied to arrays that are not of the same shape, but only if the shapes satisfy certain conditions.

In [3]:
# Add two arrays with different shapes

a = np.array([[1.],
              [2.],
              [3.],
              [4.]])  # shape (4, 1)

b = np.array([0., 1., 2.])  # shape (3,) 

a + b

array([[1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [4., 5., 6.]])

This is the addition

    [ [1.],    +  [0., 1., 2.]  
      [2.],  
      [3.],  
      [4.] ]

To execute it, numpy:
1. Aligned the shapes of `a` and `b` on the last axis and prepended 1s to the shape with fewer axes:
        a: 4 x 1     --->    a: 4 x 1
        b:     3     --->    b: 1 x 3
        

2. Checked that the sizes of the axes matched or were equal to 1:
        a: 4 x 1  
        b: 1 x 3
`a` and `b` satisfied this criterion. 


3. Stretched both arrays on their 1-valued axes so that their shapes matched, then added them together.  
`a` was replicated 3 times in the second axis, while `b` was replicated 4 times in the first axis.

This meant that the addition in the final step was

    [ [1., 1., 1.],    +  [ [0., 1., 2.],  
      [2., 2., 2.],         [0., 1., 2.],  
      [3., 3., 3.],         [0., 1., 2.],  
      [4., 4., 4.] ]        [0., 1., 2.] ]
      
Addition was then carried out element-by-element, as verified by referring back to the output of the code cell above.  
This resulted in an output with shape 4 x 3.


## Numpy's broadcasting rule

Brodcasting rules for numpy:
> Prepend 1s to the smaller shape,   
check that the axes of both arrays have sizes that are equal or 1,  
then stretch the arrays in their size-1 axes.

**Note: does not require the input arrays have the same number of axes:**

        a: 3 x 7 x 1  
        b:     1 x 5  
    a * b: 3 x 7 x 5


In [8]:
a = np.array([[[0.01], [0.1]],
              [[1.00], [10.]]])  # shape (2, 2, 1)
b = np.array([[[2., 2.]],
              [[3., 3.]]])       # shape (2, 1, 2)

a * b # shape (2, 2, 2)

array([[[2.e-02, 2.e-02],
        [2.e-01, 2.e-01]],

       [[3.e+00, 3.e+00],
        [3.e+01, 3.e+01]]])

In [9]:
a = np.array([-1., 0., 1.])
b = np.array([0., 1., 2., 3.])

a[:, np.newaxis] * b  # outer product ab^T, where a and b are column vectors

array([[-0., -1., -2., -3.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  1.,  2.,  3.]])

In [12]:
a = [[1.], [2.], [3.]]
b = np.zeros(shape=[10, 1, 1])
c = np.ones(shape=[4])

In [13]:
# shape before executing this cell

(a + b).shape

(10, 3, 1)

In [14]:
# shape before executing this cell

(a*c).shape

(3, 4)

In [15]:
# shape before executing this cell

(a*b + c).shape

(10, 3, 4)

## Broadcasting for univariate TensorFlow Distributions

The broadcasting rule for TensorFlow is the same as for numpy.

In [16]:
# Define a batch of Normal distributions without broadcasting:

batch_of_normals = tfd.Normal(loc=[0., 0., 0., 1., 1., 1.], scale=[1., 10., 100., 1., 10., 100.])
batch_of_normals

In [17]:
# Parameter values for loc:

batch_of_normals.loc

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0., 0., 0., 1., 1., 1.], dtype=float32)>

In [18]:
# Parameter values for scale

batch_of_normals.scale

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([  1.,  10., 100.,   1.,  10., 100.], dtype=float32)>

A more succinct way to create a batch of distributions for this parameter grid is to use broadcasting: 
    
    loc = [ [0.],
            [1.] ]
    scale = [1., 10., 100.]
    
Shapes:

    loc:   2 x 1 ---> 2 x 3
    scale: 1 x 3 ---> 2 x 3
    
resulting in

    loc = [ [0., 0., 0.],
            [1., 1., 1.] ]
    scale = [ [1., 10., 100.],
              [1., 10., 100.] ]
              

In [19]:
# Batch of Normal distributions with broadcasting:

loc = [[0.],
       [1.]]
scale = [1., 10., 100.]

another_batch_of_normals = tfd.Normal(loc=loc, scale=scale)
another_batch_of_normals

In [20]:
another_batch_of_normals.loc

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [1.]], dtype=float32)>

In [21]:
another_batch_of_normals.scale

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([  1.,  10., 100.], dtype=float32)>

#### Broadcasting with `prob` and `log_prob` methods

In [27]:
# Batch of Normal distributions with broadcasting

loc = [[0.],
       [10.]]
scale = [1., 1., 1.]

another_batch_of_normals = tfd.Normal(loc=loc, scale=scale)
another_batch_of_normals

<tfp.distributions.Normal 'Normal' batch_shape=[2, 3] event_shape=[] dtype=float32>

In [28]:

sample = tf.random.uniform((2,1))
another_batch_of_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2.618691e-01, 2.618691e-01, 2.618691e-01],
       [5.308663e-19, 5.308663e-19, 5.308663e-19]], dtype=float32)>

In [29]:

sample = tf.random.uniform((1,3))
another_batch_of_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[3.6396638e-01, 3.6047989e-01, 3.3060524e-01],
       [5.0906424e-21, 6.2768311e-21, 2.9299765e-20]], dtype=float32)>

In [30]:

sample = tf.random.uniform((1,1))
another_batch_of_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2.4269545e-01, 2.4269545e-01, 2.4269545e-01],
       [1.0006319e-18, 1.0006319e-18, 1.0006319e-18]], dtype=float32)>

`log_prob` works in the exact same way with broadcasting. We can replace `prob` with `log_prob` in any of the previous examples:

In [31]:

sample = tf.random.uniform((1,3))
another_batch_of_normals.log_prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.1625737 ,  -1.2653036 ,  -0.97880614],
       [-44.1821    , -42.94226   , -47.51853   ]], dtype=float32)>

## Broadcasting for multivariate TensorFlow distributions

In [32]:
# Multivariate Gaussian distribution without broadcasting

single_mvt_normal = tfd.MultivariateNormalDiag(loc=[0., 0.], scale_diag=[1., 0.5])
single_mvt_normal

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[] event_shape=[2] dtype=float32>

In [33]:

single_mvt_normal.loc

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [34]:
# Covariance matrix

single_mvt_normal.covariance()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.  , 0.  ],
       [0.  , 0.25]], dtype=float32)>

The size of the final axis of the inputs determines the event shape for each distribution in the batch.  This means that if we pass
    
    loc = [ [0., 0.],
            [1., 1.] ]
    scale_diag = [1., 0.5]
    
such that

    loc:        2 x 2
    scale_diag: 1 x 2
                    ^ final dimension is interpreted as event dimension
                ^ other dimensions are interpreted as batch dimensions  
then a batch of two bivariate normal distributions will be created.

In [36]:
# Multivariate Gaussian distribution with broadcasting

loc = [ [0., 0.],
        [1., 1.] ]
scale_diag = [1., 0.5]

batch_of_mvt_normals = tfd.MultivariateNormalDiag(loc=loc, scale_diag=scale_diag)
batch_of_mvt_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2] event_shape=[2] dtype=float32>

In [37]:
# Print the distribution parameters

batch_of_mvt_normals.parameters

{'loc': ListWrapper([ListWrapper([0.0, 0.0]), ListWrapper([1.0, 1.0])]),
 'scale_diag': ListWrapper([1.0, 0.5]),
 'scale_identity_multiplier': None,
 'validate_args': False,
 'allow_nan_stats': True,
 'name': 'MultivariateNormalDiag'}


Align the parameter array shapes on their last axis, prepending 1s where necessary:  
    
           loc: 1 x 2 x 3  
    scale_diag: 2 x 1 x 3  

The final axis has size 3, so `event_shape = (3)`. The remaining axes are broadcast over to yield  
    
           loc: 2 x 2 x 3  
    scale_diag: 2 x 2 x 3  

so `batch_shape = (2, 2)`.


In [39]:
# Multivariate Gaussian distribution with broadcasting

loc = [ [ 1.,  1.,  1.],
        [-1., -1., -1.] ]  # shape (2, 3)
scale_diag = [ [[0.1, 0.1, 0.1]],
               [[10., 10., 10.]] ]  # shape (2, 1, 3)

another_batch_of_mvt_normals = tfd.MultivariateNormalDiag(loc=loc, scale_diag=scale_diag)
another_batch_of_mvt_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2, 2] event_shape=[3] dtype=float32>

In [40]:
# Print the distribution parameters

another_batch_of_mvt_normals.parameters

{'loc': ListWrapper([ListWrapper([1.0, 1.0, 1.0]), ListWrapper([-1.0, -1.0, -1.0])]),
 'scale_diag': ListWrapper([ListWrapper([ListWrapper([0.1, 0.1, 0.1])]), ListWrapper([ListWrapper([10.0, 10.0, 10.0])])]),
 'scale_identity_multiplier': None,
 'validate_args': False,
 'allow_nan_stats': True,
 'name': 'MultivariateNormalDiag'}

In [41]:
# Batch of Normal distributions with broadcasting

loc = [[0.],
       [1.],
       [0.]]
scale = [1., 10., 100., 1., 10, 100.]

another_batch_of_normals = tfd.Normal(loc=loc, scale=scale)
another_batch_of_normals

<tfp.distributions.Normal 'Normal' batch_shape=[3, 6] event_shape=[] dtype=float32>

Use `Independent` to roll the rightmost batch shape into the event shape. 

In [42]:
# Multivariate Independent distribution

another_batch_of_mvt_normals = tfd.Independent(another_batch_of_normals)
another_batch_of_mvt_normals

<tfp.distributions.Independent 'IndependentNormal' batch_shape=[3] event_shape=[6] dtype=float32>

In [45]:
# Use broadcasting with the prob method (broadcast over event) - B shaped

sample = tf.random.uniform((3, 1))
another_batch_of_mvt_normals.prob(sample)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.347357e-09, 2.621434e-09, 2.871437e-09], dtype=float32)>

In [46]:
# Use broadcasting with the prob method (broadcast over batch) - E shaped

sample = tf.random.uniform((1, 6))
another_batch_of_mvt_normals.prob(sample)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.8638230e-09, 2.0166586e-09, 3.8638230e-09], dtype=float32)>

In [47]:
# Use broadcasting with the prob method (broadcast over samples) - [S,B,E] shaped

sample = tf.random.uniform((2, 3, 6))
another_batch_of_mvt_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2.6432239e-09, 2.8803837e-09, 3.0818599e-09],
       [2.5848172e-09, 2.8024925e-09, 3.5748973e-09]], dtype=float32)>

In [48]:
# [S,b,e] shaped input where [b,e] can be broadcast agaisnt [B,E]
sample = tf.random.uniform((2, 1, 6))
another_batch_of_mvt_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[3.6079557e-09, 2.1770494e-09, 3.6079557e-09],
       [2.9538969e-09, 3.2519281e-09, 2.9538969e-09]], dtype=float32)>

In [49]:
# Use broadcasting with the log_prob method
# [S,b,e] shaped input where [b,e] can be broadcast agaisnt [B,E]

sample = tf.random.uniform((2, 3, 1))
another_batch_of_mvt_normals.prob(sample)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[3.1677998e-09, 4.0128998e-09, 2.7438039e-09],
       [2.1045983e-09, 2.3936950e-09, 3.1428360e-09]], dtype=float32)>